In [1]:
from g4beam import *
from scan import *

# Import necessary packages
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import cm
import numpy as np
import pandas as pd
from tqdm import *
import pickle
import itertools
from tabulate import tabulate
import scipy.optimize
import lzma

# Parameter list
PARAMS = [
    "t_emit",
    "momentum",
    "beta",
    "alpha",
    "l_emit",
    "pz_std",
    "vd_dist",
    "w1_length",
    "w1_angle",
    "w2_length",
    "w2_angle",
    "drift_length",
    "rf_freq",
    "rf_phase",
    "rf_length",
    "rf_grad"
]

# Helper function to load parameter sets
def load_params(filename):
    with open(filename, "rb+") as file:
        parameters = pickle.load(file)
        globals().update(parameters)
        print("Loaded from", filename)

# Load up the 145 μm optimized parameters as a test
load_params("results/parameters/145_new.pkl")

Loaded from results/parameters/145_new.pkl


In [2]:
particleCountRange = np.geomspace(1e4, 1e6, 30).astype(int)
maxStepRange = np.geomspace(0.01, 10, 30)
wedgeStepRange = np.geomspace(0.01, 10, 30)

In [3]:
def fun(N):
    pre_w1 = gen_distribution((beta, alpha, t_emit, 0, 0), (beta, alpha, t_emit, 0, 0), momentum, pz_std, z_emit=l_emit, N=N)
    pre_w1["PDGid"] = -13
    start_time = time.time()
    post_w1 = run_distribution(pre_w1, w1_length, w1_angle, vd_dist, axis=0)
    return post_w1, time.time()-start_time

results = run_scan(fun, (particleCountRange,), "results/accuracy_particleCount.pkl.lzma", trials=30)

  2%|█▊                                                                              | 20/900 [03:21<5:26:33, 22.27s/it]
Stack Trace for signal 2:
0             0x746f2a  g4bl  (null) + 7630634
1       0x7fa319d7fdb0  libc.so.6  (null) + 140338489982384
2            0x13fda0d  g4bl  (null) + 20961805
3            0x1404288  g4bl  (null) + 20988552
4            0x14051b8  g4bl  (null) + 20992440
5            0x140a099  g4bl  (null) + 21012633
6            0x140a921  g4bl  (null) + 21014817
7            0x1431737  g4bl  (null) + 21174071
8             0xe4d86e  g4bl  (null) + 14997614
9             0xa70125  g4bl  (null) + 10944805
10            0xa6e720  g4bl  (null) + 10938144
11            0xa65b0b  g4bl  (null) + 10902283
12            0x9f8582  g4bl  (null) + 10454402
13            0x744469  g4bl  (null) + 7619689
14            0x9b7cfe  g4bl  (null) + 10190078
15            0x72b123  g4bl  (null) + 7516451
16            0x556d87  g4bl  (null) + 5598599
17      0x7fa319d6aeb0  libc.

KeyboardInterrupt: 

In [6]:
pre_w1 = gen_distribution((beta, alpha, t_emit, 0, 0), (beta, alpha, t_emit, 0, 0), momentum, pz_std, z_emit=l_emit)
pre_w1["PDGid"] = -13
run_distribution(pre_w1, w1_length, w1_angle, vd_dist, axis=0, debug=True)

abshalfangle3=46.538398375036195 absoffset3=3.5547471253984786 abshgt=11.472300160066446 absLEN3=24 wedgeAxis=0


In [5]:
with lzma.open("results/accuracy_particleCount.pkl.lzma", "rb") as file:
    results = pickle.load(file)

fig, ((ax1, ax2, ax3, ax4, ax5, ax6)) = plt.subplots(nrows=6, layout="constrained")
fig.set_size_inches(12, 6)
ax1.errorbar(*qmap_to_arrays(maxSteps, calc_quantity(lambda x: emittances(cut_outliers(x[0]))[0], results)), capsize=2)
ax1.set_xlabel(r"N")
ax1.set_ylabel(r"x emit (mm)")
ax1.set_yscale("log")

ax2.errorbar(*qmap_to_arrays(maxSteps, calc_quantity(lambda x: emittances(cut_outliers(x[0]))[1], results)), capsize=2)
ax2.set_xlabel(r"N")
ax2.set_ylabel(r"y emit (mm)")
ax2.set_yscale("log")

ax3.errorbar(*qmap_to_arrays(maxSteps, calc_quantity(lambda x: emittances(cut_outliers(x[0]))[2], results)), capsize=2)
ax3.set_xlabel(r"N")
ax3.set_ylabel(r"z emit (mm)")
ax3.set_yscale("log")

ax4.errorbar(*qmap_to_arrays(maxSteps, calc_quantity(lambda x: np.std(p_total(cut_outliers(x[0]))), results)), capsize=2)
ax4.set_xlabel(r"N")
ax4.set_ylabel(r"$\sigma_p$ (MeV/c)")
ax4.set_yscale("log")

ax5.errorbar(*qmap_to_arrays(maxSteps, calc_quantity(lambda x: np.mean(p_total(cut_outliers(x[0]))), results)), capsize=2)
ax5.set_xlabel(r"N")
ax5.set_ylabel(r"mean momentum (MeV/c)")
ax5.set_yscale("log")

ax6.errorbar(*qmap_to_arrays(maxSteps, calc_quantity(lambda x: x[1], results)), capsize=2)
ax6.set_xlabel(r"N")
ax6.set_ylabel(r"time")
ax6.set_yscale("log")

EOFError: Compressed file ended before the end-of-stream marker was reached